# 02 – Preprocessing Komentar (Cleaning & Label Encoding)

Notebook ini melakukan **preprocessing** pada dataset komentar YouTube
terkait isu dugaan korupsi proyek Kereta Cepat Whoosh.

Dataset input:
- `data/labeled_dataset_whoosh.csv`  → hasil labeling + koreksi manual

Output utama:
- `data/clean_comments.csv` → teks sudah dibersihkan + label terenkripsi (0/1/2)

Tahapan:
1. Load dataset final berlabel
2. Normalisasi label (positive / negative / neutral)
3. Menghapus data yang tidak dipakai (label kosong, komentar duplikat, dsb.)
4. Cleaning teks:
   - lowercase
   - hapus URL, mention, hashtag
   - hapus angka & tanda baca berlebihan
   - normalisasi spasi & huruf berulang
   - normalisasi kata-kata alay/singkat (sebagian)
5. Menambahkan kolom `text_clean` dan `label_encoded`
6. Simpan dataset hasil preprocessing


## Import Library

In [4]:
import os
import re
import unicodedata

import pandas as pd
from tqdm.auto import tqdm


c:\Users\hafizh\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load Dataset

In [7]:
DATA_DIR = "../data"
FINAL_PATH = os.path.join(DATA_DIR, "labeled_dataset_whoosh.csv")

df = pd.read_csv(FINAL_PATH)
print("Jumlah baris dalam labeled_dataset_whoosh.csv:", len(df))
df.head()

Jumlah baris dalam labeled_dataset_whoosh.csv: 1000


,video_id,video_title,comment_id,author,comment,likes,published_at,sentiment
0,1_Xrj0mb7K4,Bedah KEGILAAN Project Whoosh,Ugy77mNqV3jn_Xf_f2x4AaABAg,@mokohardono8859,Yg benci ya apa aja salah.. \nYg seneng ya mak...,0,2025-12-03T05:33:53Z,neutral
1,1_Xrj0mb7K4,Bedah KEGILAAN Project Whoosh,UgzY1qxEeVrMeskZ39V4AaABAg,@Muhidin13-kr1zl,Bandung akan miliki kereta pajajaran dgn beaya...,0,2025-12-02T09:48:07Z,neutral
2,1_Xrj0mb7K4,Bedah KEGILAAN Project Whoosh,UgzlM3ejBnN9PKqB2sB4AaABAg,@MA_Alpha-l4q,SUDAH JELAS GENG SOLO YANG HARUS BERTANGGUNG J...,1,2025-12-01T06:59:37Z,negative
3,1_Xrj0mb7K4,Bedah KEGILAAN Project Whoosh,UgzcxKsZ3V_n222CvEF4AaABAg,@nurhasanahssi2114,"Jokowi, Luhut, kroni2 yg harus bertanggungjaw...",0,2025-11-30T01:18:59Z,negative
4,1_Xrj0mb7K4,Bedah KEGILAAN Project Whoosh,Ugx1UtqOkCAflakGpoh4AaABAg,@omsimon-k6k,Yg ditangkap gorengan yg makan duduk manis,0,2025-11-28T15:56:25Z,neutral


## Cek Kolom Penting & Bersihkan Label Kosong

In [9]:
if "comment" not in df.columns:
    raise ValueError("Kolom 'comment' tidak ditemukan di labeled_dataset_whoosh.csv")

if "sentiment" not in df.columns:
    raise ValueError("Kolom 'sentiment' tidak ditemukan di labeled_dataset_whoosh.csv")

df[["comment", "sentiment"]].head()


,comment,sentiment
0,Yg benci ya apa aja salah.. \nYg seneng ya mak...,neutral
1,Bandung akan miliki kereta pajajaran dgn beaya...,neutral
2,SUDAH JELAS GENG SOLO YANG HARUS BERTANGGUNG J...,negative
3,"Jokowi, Luhut, kroni2 yg harus bertanggungjaw...",negative
4,Yg ditangkap gorengan yg makan duduk manis,neutral


## Normalisasi nilai label (antisipasi huruf besar/kecil & bahasa campur)

In [11]:

df["sentiment"] = df["sentiment"].astype(str).str.strip().str.lower()

label_map_normalize = {
    "positif": "positive",
    "positif ": "positive",
    "positif👍": "positive",
    "positive": "positive",

    "negatif": "negative",
    "negatif ": "negative",
    "negative": "negative",

    "netral": "neutral",
    "neutral": "neutral",
}

df["sentiment"] = df["sentiment"].map(label_map_normalize).fillna(df["sentiment"])

df["sentiment"].value_counts(dropna=False)

sentiment
negative    646
neutral     320
positive     34
Name: count, dtype: int64

## Buang baris yang label-nya tidak valid

In [12]:
valid_labels = {"positive", "negative", "neutral"}

mask_valid = df["sentiment"].isin(valid_labels)
print("Baris valid:", mask_valid.sum())
print("Baris tidak valid (akan dibuang):", (~mask_valid).sum())

df = df[mask_valid].reset_index(drop=True)
df["sentiment"].value_counts()

Baris valid: 1000
Baris tidak valid (akan dibuang): 0


sentiment
negative    646
neutral     320
positive     34
Name: count, dtype: int64

## Buang Duplikat & Komentar Kosong / Pendek

In [13]:
# Hapus baris dengan komentar kosong/null
df["comment"] = df["comment"].astype(str)
df["comment_stripped"] = df["comment"].str.strip()

before = len(df)
df = df[df["comment_stripped"] != ""].reset_index(drop=True)
after = len(df)

print(f"Menghapus komentar kosong: {before - after} baris dihapus. Sisa: {after}")

Menghapus komentar kosong: 0 baris dihapus. Sisa: 1000


In [14]:
# Hapus duplikat berdasarkan teks komentar (dan comment_id kalau ada)
subset_cols = ["comment_stripped"]
if "comment_id" in df.columns:
    subset_cols.append("comment_id")

before = len(df)
df = df.drop_duplicates(subset=subset_cols).reset_index(drop=True)
after = len(df)

print(f"Menghapus duplikat: {before - after} baris dihapus. Sisa: {after}")

Menghapus duplikat: 0 baris dihapus. Sisa: 1000


In [15]:
# (Opsional) Hapus komentar yang terlalu pendek (misal <= 2 kata)
def count_words(text: str) -> int:
    return len(str(text).split())

df["word_count"] = df["comment_stripped"].apply(count_words)

before = len(df)
df = df[df["word_count"] > 2].reset_index(drop=True)
after = len(df)

print(f"Menghapus komentar sangat pendek (<=2 kata): {before - after} baris dihapus. Sisa: {after}")

Menghapus komentar sangat pendek (<=2 kata): 47 baris dihapus. Sisa: 953


## Fungsi Cleaning Teks Bahasa Indonesia

In [32]:
# Beberapa kamus normalisasi singkat / alay (bisa kamu tambah sendiri)
slang_dict = {
    "gak": "tidak",
    "ga": "tidak",
    "nggak": "tidak",
    "ngga": "tidak",
    "gk": "tidak",
    "kagak": "tidak",
    "aja": "saja",
    "ajaib": "ajaib",  # contoh; bisa diubah
    "bgt": "banget",
    "bngt": "banget",
    "sm": "sama",
    "tp": "tapi",
    "klo": "kalau",
    "klu": "kalau",
    "kl": "kalau",
    "yg": "yang",
    "dr": "dari",
    "jd": "jadi",
    "aja": "saja",
    "udh": "sudah",
    "udah": "sudah",
    "dah": "sudah",
    "btw": "ngomong-ngomong",
    "yg": "yang",
    "tdk": "tidak",
    "tak": "tidak",
    "dgn": "dengan",
    "dg": "dengan",
    "kalo": "kalau",
    "sdh": "sudah",
    "sy": "saya",
    "utk": "untuk",
    "krn": "karna",
    "dpt": "dapat",
    "y": "ya"
}

In [28]:
def normalize_unicode(text: str) -> str:
    # Hilangkan karakter aneh / kombinasi
    return unicodedata.normalize("NFKC", text)

In [29]:
URL_PATTERN = re.compile(r'https?://\S+|www\.\S+')
MENTION_PATTERN = re.compile(r'@\w+')
HASHTAG_PATTERN = re.compile(r'#\w+')
NON_ALPHANUM_PATTERN = re.compile(r'[^a-zA-Z0-9\s]')
MULTISPACE_PATTERN = re.compile(r'\s+')

In [36]:
def clean_text(text: str) -> str:
    text = str(text)

    # 1. Normalisasi unicode
    text = normalize_unicode(text)

    # 2. Lowercase
    text = text.lower()

    # 3. Hapus URL, mention, hashtag
    text = URL_PATTERN.sub(" ", text)
    text = MENTION_PATTERN.sub(" ", text)
    text = HASHTAG_PATTERN.sub(" ", text)

    # 4. Hapus angka dan tanda baca non-alfanumerik (opsional: angka bisa dibiarkan)
    text = NON_ALPHANUM_PATTERN.sub(" ", text)

    # 5. Hilangkan huruf berulang (contoh: "parahhh" -> "parahh", "wkwkwk" tetap dibiarkan seadanya)
    text = re.sub(r'(.)\1{2,}', r'\1\1', text)

    # 6. Normalisasi slang sederhana
    tokens = text.split()
    tokens_norm = [slang_dict.get(tok, tok) for tok in tokens]
    text = " ".join(tokens_norm)

    # 7. Hilangkan spasi berlebih
    text = MULTISPACE_PATTERN.sub(" ", text).strip()

    # 8. Hapus angka
    text = re.sub(r'\d+', ' ', text) 

    return text

In [37]:
sample = df["comment"].iloc[0]
print("Sebelum :", sample)
print("Sesudah :", clean_text(sample))

Sebelum : Yg benci ya apa aja salah.. 
Yg seneng ya makin seneng... 
Yg nyinyir y dpt bahan.. 
Yg dukung y anggap kesuksesan.. 
Terserah rakyat nusantara aja...
Sesudah : yang benci ya apa saja salah yang seneng ya makin seneng yang nyinyir ya dapat bahan yang dukung ya anggap kesuksesan terserah rakyat nusantara saja


## Apply Cleaning ke Semua Komentar

In [38]:
tqdm.pandas(desc="Membersihkan teks")

df["text_clean"] = df["comment_stripped"].progress_apply(clean_text)

df[["comment", "text_clean", "sentiment"]].head()


Membersihkan teks: 100%|██████████| 953/953 [00:00<00:00, 48902.84it/s]


,comment,text_clean,sentiment
0,Yg benci ya apa aja salah.. \nYg seneng ya mak...,yang benci ya apa saja salah yang seneng ya ma...,neutral
1,Bandung akan miliki kereta pajajaran dgn beaya...,bandung akan miliki kereta pajajaran dengan be...,neutral
2,SUDAH JELAS GENG SOLO YANG HARUS BERTANGGUNG J...,sudah jelas geng solo yang harus bertanggung j...,negative
3,"Jokowi, Luhut, kroni2 yg harus bertanggungjaw...",jokowi luhut kroni yang harus bertanggungjawa...,negative
4,Yg ditangkap gorengan yg makan duduk manis,yang ditangkap gorengan yang makan duduk manis,neutral


## Label Encoding (0 = negative, 1 = neutral, 2 = positive)

In [24]:
label_to_int = {
    "negative": 0,
    "neutral": 1,
    "positive": 2,
}

df["label_encoded"] = df["sentiment"].map(label_to_int)

df["label_encoded"].value_counts().sort_index()

label_encoded
0    628
1    292
2     33
Name: count, dtype: int64

In [25]:
mapping_df = pd.DataFrame([
    {"label_text": k, "label_encoded": v} for k, v in label_to_int.items()
])
mapping_df

,label_text,label_encoded
0,negative,0
1,neutral,1
2,positive,2


In [40]:
OUTPUT_PATH = os.path.join(DATA_DIR, "clean_comments.csv")

cols_to_keep = [
    "comment",        # teks asli
    "text_clean",     # teks yang sudah dibersihkan
    "sentiment",      # label teks
    "label_encoded",  # label numerik
]

df_clean = df[cols_to_keep].copy()
df_clean.to_csv(OUTPUT_PATH, index=False, encoding="utf-8")

print(f"Dataset bersih disimpan ke: {OUTPUT_PATH}")
df_clean.head()

Dataset bersih disimpan ke: ../data\clean_comments.csv


,comment,text_clean,sentiment,label_encoded
0,Yg benci ya apa aja salah.. \nYg seneng ya mak...,yang benci ya apa saja salah yang seneng ya ma...,neutral,1
1,Bandung akan miliki kereta pajajaran dgn beaya...,bandung akan miliki kereta pajajaran dengan be...,neutral,1
2,SUDAH JELAS GENG SOLO YANG HARUS BERTANGGUNG J...,sudah jelas geng solo yang harus bertanggung j...,negative,0
3,"Jokowi, Luhut, kroni2 yg harus bertanggungjaw...",jokowi luhut kroni yang harus bertanggungjawa...,negative,0
4,Yg ditangkap gorengan yg makan duduk manis,yang ditangkap gorengan yang makan duduk manis,neutral,1


## Ringkasan Preprocessing

Langkah-langkah yang sudah dilakukan di notebook ini:

1. Load `labeled_dataset_whoosh.csv` (label hasil Gemini).
2. Normalisasi label menjadi tiga kelas:
   - `positive`
   - `negative`
   - `neutral`
3. Menghapus:
   - komentar kosong,
   - komentar duplikat,
   - komentar yang terlalu pendek.
4. Melakukan cleaning teks:
   - lowercase,
   - hapus URL, mention, hashtag,
   - hapus karakter non-alfanumerik,
   - normalisasi spasi & huruf berulang,
   - normalisasi beberapa kata slang.
5. Menambahkan:
   - `text_clean` → teks bersih,
   - `label_encoded` → 0 (negatif), 1 (netral), 2 (positif).
6. Menyimpan dataset hasil preprocessing ke:
   - `data/clean_comments.csv`

Notebook berikutnya:
- `03_eda.ipynb` → analisis distribusi sentimen, wordcloud, tf-idf, dsb.
- `04_baseline_models.ipynb` → training Logistic Regression, SVM, Naive Bayes.
- `05_indobert_training.ipynb` → fine-tuning IndoBERT.

In [1]:
import pandas as pd
datapre = pd.read_csv("../data/clean_comments.csv")
datapre

,comment,text_clean,sentiment,label_encoded
0,Yg benci ya apa aja salah.. \nYg seneng ya mak...,yang benci ya apa saja salah yang seneng ya ma...,neutral,1
1,Bandung akan miliki kereta pajajaran dgn beaya...,bandung akan miliki kereta pajajaran dengan be...,neutral,1
2,SUDAH JELAS GENG SOLO YANG HARUS BERTANGGUNG J...,sudah jelas geng solo yang harus bertanggung j...,negative,0
3,"Jokowi, Luhut, kroni2 yg harus bertanggungjaw...",jokowi luhut kroni yang harus bertanggungjawa...,negative,0
4,Yg ditangkap gorengan yg makan duduk manis,yang ditangkap gorengan yang makan duduk manis,neutral,1
...,...,...,...,...
948,Sekumpulang orang2 berkebutuhan khusus 😂😂😂😂.,sekumpulang orang berkebutuhan khusus,neutral,1
949,Lanjutkan pak said...,lanjutkan pak said,neutral,1
950,Kenapa Mahfud MD memberikan Pembelaan bahwa Lu...,kenapa mahfud md memberikan pembelaan bahwa lu...,negative,0
951,Dosa jokowi sudah terlalu banyak sama rakyat k...,dosa jokowi sudah terlalu banyak sama rakyat k...,negative,0
